In [2]:
#@title License information { display-mode: "form" }
#@markdown Copyright 2024 The MediaPipe Authors.
#@markdown Licensed under the Apache License, Version 2.0 (the "License");
#@markdown
#@markdown you may not use this file except in compliance with the License.
#@markdown You may obtain a copy of the License at
#@markdown
#@markdown https://www.apache.org/licenses/LICENSE-2.0
#@markdown
#@markdown Unless required by applicable law or agreed to in writing, software
#@markdown distributed under the License is distributed on an "AS IS" BASIS,
#@markdown WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#@markdown See the License for the specific language governing permissions and
#@markdown limitations under the License.

In [3]:
#@title Setup { display-mode: "form" }
!pip install --upgrade numpy


import ipywidgets as widgets
from IPython.display import display
from google.colab import files
install_out = widgets.Output()
display(install_out)
with install_out:
  !pip install --upgrade numpy
  !pip install --upgrade scipy
  !pip install mediapipe
  from mediapipe.tasks.python.genai import bundler

install_out.clear_output()
with install_out:
  print("Setup done.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 32.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.


Output()

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
tokenizer.save_pretrained("bert_tokenizer/")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

('bert_tokenizer/tokenizer_config.json',
 'bert_tokenizer/special_tokens_map.json',
 'bert_tokenizer/vocab.txt',
 'bert_tokenizer/added_tokens.json',
 'bert_tokenizer/tokenizer.json')

In [5]:
# build_fake_corpus.py
with open("bert_tokenizer/vocab.txt", "r") as fin, open("fake_corpus.txt", "w") as fout:
    for line in fin:
        token = line.strip()
        if token and not token.startswith("["):
            fout.write(token.replace("##", "") + "\n")


In [6]:
!pip install sentencepiece


In [18]:
# !pip install tf-keras # Install the compatible Keras for transformers
# !pip install --upgrade tensorflow # Ensure TensorFlow is upgraded
# !pip install tensorflow --no-deps # Reinstall TensorFlow without dependencies
from transformers import TFAutoModelForQuestionAnswering
import tensorflow as tf # Import tensorflow

# Instead of importing from tf_keras, import from tensorflow.keras.activations
from tensorflow.keras import activations

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad")
model.save("distilbert_tf_model")

converter = tf.lite.TFLiteConverter.from_saved_model("distilbert_tf_model") # Use tf.lite
tflite_model = converter.convert()

# Save the model to a file
with open("distilbert-qa.tflite", "wb") as f:
    f.write(tflite_model)

All PyTorch model weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [8]:
!find / -name "spm_train" 2>/dev/null

In [9]:
!apt-get update && apt-get install -y cmake build-essential pkg-config libgoogle-perftools-dev
!git clone https://github.com/google/sentencepiece.git
%cd sentencepiece
!mkdir build && cd build && cmake .. && make -j $(nproc) && make install && ldconfig
%cd /content

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,788 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,243 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Pack

In [10]:
!spm_train --version

/bin/bash: line 1: spm_train: command not found


In [11]:
!spm_train \
      --input=fake_corpus.txt \
      --model_prefix=distilbert_tokenizer \
      --vocab_size=26598 \
      --character_coverage=1.0 \
      --model_type=word

/bin/bash: line 1: spm_train: command not found


In [12]:
#!pip install --upgrade --force-reinstall mediapipe-model-maker==0.2.1.4
#!pip install --upgrade  --user mediapipe-model-maker==0.2.1.4
!python3 -m mediapipe_model_maker.genai.task_converter \
  --tflite_model distilbert-qa.tflite \
  --tokenizer_model distilbert_tokenizer.model \
  --output_file distilbert_qa.task

/usr/bin/python3: Error while finding module specification for 'mediapipe_model_maker.genai.task_converter' (ModuleNotFoundError: No module named 'mediapipe_model_maker')


In [13]:
!pip install mediapipe
!pip install --upgrade mediapipe-model-maker # Install or upgrade mediapipe-model-maker
!pip install sentencepiece  # Install or upgrade sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.

In [14]:
#!pip install tensorflow text
#!pip install tensorflow_hub
#!pip uninstall mediapipe-model-maker
#!pip install --upgrade mediapipe-model-maker

# !pip uninstall mediapipe-model-maker
# !pip install --upgrade --force-reinstall mediapipe-model-maker==0.2.1.4

!find / -name "mediapipe_model_maker" 2>/dev/null
!find / -name "model_maker" 2>/dev/null


import site
import os

for path in site.getsitepackages():
    package_path = os.path.join(path, 'mediapipe_model_maker')
    if os.path.exists(package_path):
        print(f"Found mediapipe_model_maker at: {package_path}")
        break  # Stop searching once found
else:
    print("mediapipe_model_maker not found in site-packages.")


!ls {package_path}
!ls {package_path}/genai  # Assuming genai subfolder exists

/usr/local/lib/python3.11/dist-packages/mediapipe_model_maker
/usr/local/lib/python3.11/dist-packages/mediapipe/model_maker
Found mediapipe_model_maker at: /usr/local/lib/python3.11/dist-packages/mediapipe_model_maker
__init__.py  __pycache__  python
ls: cannot access '/usr/local/lib/python3.11/dist-packages/mediapipe_model_maker/genai': No such file or directory


In [15]:
#import tensorflow as tf; print(tf.__version__)

!ls /usr/local/lib/python3.11/dist-packages/mediapipe_model_maker
!ls /usr/local/lib/python3.11/dist-packages/mediapipe_model_maker/genai

__init__.py  __pycache__  python
ls: cannot access '/usr/local/lib/python3.11/dist-packages/mediapipe_model_maker/genai': No such file or directory


# Why do we need task bundles?

Executing a text generation pipeline in its entirety requires more than merely employing the core transformer model. It requires preparing the input text to align with the model's required format, running the model autoregressively, and sampling during each iteration. Consequently, once the user has converted their model into `tflite` format (comprising the model's graph and parameters), it becomes essential to augment it with additional metadata to ensure successful end-to-end execution of the model.

Task bundler offers a practical approach to creating such bundles. The example below illustrates how a task bundle can be generated for a converted Gemma model.

In [17]:
from mediapipe.tasks.python.genai import bundler

tflite_model="distilbert-qa.tflite" # @param {type:"string"}
tokenizer_model="distilbert_tokenizer.model" # @param {type:"string"}
start_token="<bos>" # @param {type:"string"}
stop_token="<eos>" # @param {type:"string"}
output_filename="distilbert.task" # @param {type:"string"}
enable_bytes_to_unicode_mapping=False # @param ["False", "True"] {type:"raw"}

config = bundler.BundleConfig(
    tflite_model=tflite_model,
    tokenizer_model=tokenizer_model,
    start_token=start_token,
    stop_tokens=[stop_token],
    output_filename=output_filename,
    enable_bytes_to_unicode_mapping=enable_bytes_to_unicode_mapping,
)
bundler.create_bundle(config)

ValueError: Failed to load tokenizer model from distilbert_tokenizer.model. Please ensure you are passing a valid SentencePiece model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Download the Bundle { display-mode: "form" }
#@markdown Run this cell to download the generated `.task` file.
files.download(output_filename)

**Notes:**

* The current task pipeline only supports SentencePiece tokenizer models.
* Certain models (e.g., phi-2) use bytes to unicode mapping. Use `enable_bytes_to_unicode_mapping` flag accordingly. Such information, including `start_token` and `stop_tokens` are often provided along with model artifacts.
* The generated output bundle file must end with `.task`. If not, `create_bundle` automatically adds the extension. Do not remove or change that.

In [ ]:
import tensorflow as tf
import numpy as np
model_path = "distilbert-qa.tflite"  # Replace with your model

interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input Tensor Details:", input_details)
print("Output Tensor Details:", output_details)

# Assuming your model has two inputs: input_ids and attention_mask
# Adjust the input shapes and data types accordingly
input_shape_ids = input_details[0]['shape']
input_data_ids = np.random.randint(0, 30522, size=input_shape_ids).astype(np.int32) # Generate random token IDs

input_shape_mask = input_details[1]['shape']
input_data_mask = np.ones(input_shape_mask).astype(np.int32) # Create an attention mask of all 1s

# Set the tensors for both inputs
interpreter.set_tensor(input_details[0]['index'], input_data_ids)
interpreter.set_tensor(input_details[1]['index'], input_data_mask)

interpreter.invoke()

output_index = output_details[0]['index']
output_data = interpreter.get_tensor(output_index)

print("Raw Output Data:", output_data)

# Add your post-processing here (if needed)
# ...

In [ ]:
import tensorflow as tf
import numpy as np
import sentencepiece as spm

# 1. Load the SentencePiece tokenizer
sp = spm.SentencePieceProcessor()
sp.load("distilbert_tokenizer.model") # Load the tokenizer model

# 2. Load the TFLite model
model_path = "distilbert-qa.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 3. Define the question and context
question = "What is the capital of France?"
context = "Paris is the capital and most populous city of France."

# 4. Tokenize the input
input_text = question + " [SEP] " + context # Add [SEP] token between question and context
input_ids = sp.encode_as_ids(input_text)
input_mask = [1] * len(input_ids) # Create attention mask

# 5. Pad the input if necessary
max_length = input_details[0]['shape'][1]  # Get the maximum sequence length
input_ids = input_ids[:max_length] + [0] * (max_length - len(input_ids)) # Pad with 0s
input_mask = input_mask[:max_length] + [0] * (max_length - len(input_mask)) # Pad with 0s

# 6. Convert to NumPy arrays and set tensors
input_ids = np.array(input_ids, dtype=np.int32).reshape(input_details[0]['shape'])
input_mask = np.array(input_mask, dtype=np.int32).reshape(input_details[1]['shape'])

interpreter.set_tensor(input_details[0]['index'], input_ids)
interpreter.set_tensor(input_details[1]['index'], input_mask)

# 7. Run inference
interpreter.invoke()

# 8. Get and process the output
start_logits = interpreter.get_tensor(output_details[0]['index'])
end_logits = interpreter.get_tensor(output_details[1]['index'])

# 9. Find the answer span
start_index = np.argmax(start_logits)
end_index = np.argmax(end_logits)

# 10. Extract the answer from the context (UPDATED)
# Decode the token IDs to get the actual text of the answer
answer_tokens = input_ids[start_index : end_index + 1]
answer = sp.decode_ids(answer_tokens.tolist())


print("Question:", question)
print("Answer:", answer)